<a href="https://colab.research.google.com/github/Saito-Taishi/HomeCredit/blob/main/lgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import os
import gc
import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

oe = preprocessing.OrdinalEncoder()


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/MyDrive/Kaggle/HomeCredit2024/data/input/csv_files"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 関数の設定

In [55]:
path_feature = "/content/drive/MyDrive/Kaggle/HomeCredit2024/data/input/"
feature_definitions = pl.read_csv(path_feature + "feature_definitions.csv")
pd.set_option("display.max_colwidth",None)
pd.set_option('display.max_rows', None)


#関数の定義
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df





def extract_A(df : pl.DataFrame) -> pl.DataFrame:
  A_array = ["case_id"]
  for col in df.columns:
    if col[-1] in "A":
      A_array.append(col)
  return A_array

def extract_L(df : pl.DataFrame) -> pl.DataFrame:
  L_array = ["case_id"]
  for col in df.columns:
    if col[-1] in "L":
      L_array.append(col)
  return L_array

def extract_P(df : pl.DataFrame) -> pl.DataFrame:
  P_array = ["case_id"]
  for col in df.columns:
    if col[-1] in "P":
      P_array.append(col)
  return P_array

def extract_D(df : pl.DataFrame) -> pl.DataFrame:
  D_array = ["case_id"]
  for col in df.columns:
    if col[-1] in "D":
      D_array.append(col)
  return D_array


def search_variable(df:pl.DataFrame, features_df=feature_definitions) :
  df_col = []
  for col in df.columns:
    df_col.append(col)
  filtered_df = features_df.filter(pl.col("Variable").is_in(df_col))
  return filtered_df.to_pandas()




def drop_array(df:pl.DataFrame):
  str_array =[]
  for col in df.columns:
    str_array.append(col)
  df = df.drop(str_array)
  return df

# ファイルのインポート

In [46]:
train_base = pl.read_csv(path + "/train/train_base.csv")

static_0 = pl.concat([
    pl.read_csv(path+ "/train/train_static_0_0.csv"),
    pl.read_csv(path + "/train/train_static_0_1.csv")
],how = "vertical_relaxed").pipe(set_table_dtypes)


static_cb = pl.concat([
    pl.read_csv(path+ "/train/train_static_cb_0.csv")
],how = "vertical_relaxed"
    ).pipe(set_table_dtypes)

df = pl.concat([static_0,static_cb],how= "align")
df = pl.concat([train_base, df], how="align")

In [37]:
test_static_0 = pl.concat([
    pl.read_csv(path+ "/test/test_static_0_0.csv"),
    pl.read_csv(path + "/test/test_static_0_1.csv"),
    pl.read_csv(path + "/test/test_static_0_2.csv"),
],
    how = "vertical_relaxed"
    ).pipe(set_table_dtypes)

test_static_cb = pl.concat([
    pl.read_csv(path+ "/test/test_static_cb_0.csv")
],
    how = "vertical_relaxed"
    ).pipe(set_table_dtypes)

df_test = pl.concat([test_static_0,test_static_cb],how= "align")

# カテゴリ変数の処理 ここあんまり自信ない

In [33]:
category_array = [
    "bankacctype_710L", "cardtype_51L", "paytype1st_925L", "paytype_783L",
    "previouscontdistrict_112M", "twobodfilling_608L", "typesuite_864L",
    "datefirstoffer_1144D", "datelastinstal40dpd_247D", "datelastunpaid_3546854D",
    "dtlastpmtallstes_4499206D", "firstclxcampaign_1125D", "firstdatedue_489D",
    "lastactivateddate_801D", "lastapplicationdate_877D", "lastapprdate_640D",
    "lastdelinqdate_224D", "lastrejectdate_50D", "lastrepayingdate_696D",
    "maxdpdinstldate_3546855D", "payvacationpostpone_4187118D", "validfrom_1069D"
]


for col in category_array:
  encorder = le.fit_transform(df[col].to_pandas())
  df = df.with_columns(pl.Series(col, encorder))


df

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,…,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,…,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,str,str,str,str,str,f64,f64
0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0.0""",3.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,1,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5,null,null,3600.0,0.0,0.0,1.0,0.0,8.0,2.0,1.0,null,null,null,null,null,null,null,null,null,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,2.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
6,0.0,null,3110.8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,0,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,nul

# ベースラインモデルの作成

In [ ]:
drop_array(df)

In [60]:
#文字列の削除




X = df.drop("target")
y = df.select(pl.col("target"))


X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.6, random_state=42)

train_data = lgb.Dataset(X_train, label = y_train)
test_data = lgb.Dataset(X_valid, label  = y_valid, reference = train_data)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    train_data,
    valid_sets = test_data,
    callbacks =[lgb.log_evaluation(50), lgb.early_stopping(10)]
)





/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


TypeError: Wrong type(DataFrame) for label.
It should be list, numpy 1-D array or pandas Series

In [57]:
X

case_id,WEEK_NUM,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,…,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_693L,pmtscount_423L,pmtssum_45A,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,…,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,0,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,…,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2,0,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,…,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3,0,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,…,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4,0,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,…,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5,0,null,null,3600.0,0.0,0.0,1.0,0.0,8.0,2.0,1.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,null,…,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
6,0,0.0,null,3110.8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,null,…,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,

In [58]:
y

target
i64
0
0
0
0
1
0
0
0
0
